In [1]:
import lighthouse as lh

lh.isochrone.get_isochrones.get_mist_isochrones()
lh.stellar_atmosphere_spectrum.get_stellar_templates.get_polynomial_coefficients_villaume2017a()

Getting MIST
Writing MIST
Extracting MIST
Deleting tar file
Compiling MIST data
Writing MIST to hdf5 database


In [2]:
import matplotlib.pyplot as plt 

import torch 

import sedpy


In [4]:
S = lh.SSP.Basic_SSP(
    lh.isochrone.MIST(),
    lh.initial_mass_function.Kroupa(),
    lh.stellar_atmosphere_spectrum.PolynomialEvaluator(),
    )

In [10]:
# need to put some safeguards in wrt commensurate coverage between isochrones and stellar libraries 

print(S.isochrone.ages)
print(S.isochrone.metallicities)

tensor([ 5.0000,  5.0500,  5.1000,  5.1500,  5.2000,  5.2500,  5.3000,  5.3500,
         5.4000,  5.4500,  5.5000,  5.5500,  5.6000,  5.6500,  5.7000,  5.7500,
         5.8000,  5.8500,  5.9000,  5.9500,  6.0000,  6.0500,  6.1000,  6.1500,
         6.2000,  6.2500,  6.3000,  6.3500,  6.4000,  6.4500,  6.5000,  6.5500,
         6.6000,  6.6500,  6.7000,  6.7500,  6.8000,  6.8500,  6.9000,  6.9500,
         7.0000,  7.0500,  7.1000,  7.1500,  7.2000,  7.2500,  7.3000,  7.3500,
         7.4000,  7.4500,  7.5000,  7.5500,  7.6000,  7.6500,  7.7000,  7.7500,
         7.8000,  7.8500,  7.9000,  7.9500,  8.0000,  8.0500,  8.1000,  8.1500,
         8.2000,  8.2500,  8.3000,  8.3500,  8.4000,  8.4500,  8.5000,  8.5500,
         8.6000,  8.6500,  8.7000,  8.7500,  8.8000,  8.8500,  8.9000,  8.9500,
         9.0000,  9.0500,  9.1000,  9.1500,  9.2000,  9.2500,  9.3000,  9.3500,
         9.4000,  9.4500,  9.5000,  9.5500,  9.6000,  9.6500,  9.7000,  9.7500,
         9.8000,  9.8500,  9.9000,  9.95

In [ ]:
ssp = S.forward(torch.tensor(0., dtype = torch.float64), torch.tensor(10., dtype  = torch.float64), torch.tensor([1.3, 2.3, 2.7]))

plt.plot(S.sas.wavelength, ssp)
plt.show()